In [ ]:
###Match based on ee_name
##Generate ee_list (a unique list ready for matching with ticker)

import numpy as np
import pandas as pd
import os 
from cleanco import basename
import time
import requests


os.chdir("D:/My Drive/Wei/Patent")
patent_raw=pd.read_csv("patent_latest_owner_dt.csv")
stock_raw=pd.read_csv("PERMNO_dt.csv")


#Generate unique EE,OR,SecurityNm value
#EE & OR
ee_name=patent_raw[['ee_name']].drop_duplicates()

ee_name=ee_name.reset_index()

ee_name=ee_name[['ee_name']]

#Processing company name (drop after the first comma, incase to long company name)
tokens=pd.DataFrame(index=range(len(ee_name)),columns=range(1))
tokens_list=[r.split(',') for r in ee_name['ee_name']]

for i in range(len(ee_name)):
    x=tokens_list[i]
    tokens.at[i,0]=x[0]

for i in range (len(ee_name)):
    ee_name.at[i,'ee_name']=basename(tokens.iloc[i,0])
    
ee_name['ticker']=''

ee_name.to_csv('ee_list.csv',index=False)

##Scraping with API and generate ee_list_other(the other ee_list includes corresponding ticker)
import numpy as np
import pandas as pd
import os 
from cleanco import basename
import time
import requests
os.chdir("D:/My Drive/Wei/Patent")

#Scrap with yahoo finance api
ee_name=pd.read_csv("ee_list.csv")
def getTicker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

start = time.time()
for i in range (0,len(ee_name)):
    try:
        company_name=ee_name.iloc[i,0]
        ee_name.at[i,'ticker']=getTicker(company_name)
    except:IndexError
    pass
    
    a=ee_name.iloc[i,1]
    print(a)
    print(i)
    time.sleep(0.5) #Avoid reaching scraping times limit per minutes
    
end = time.time()
print(end - start)
ee_name.to_csv('ee_list_other.csv',index=False)

In [ ]:
##Further cleanning on the data:1.Generate ee_matcher 2.generate ee_ticlist.txt
import numpy as np
import pandas as pd
import time
import os 
os.chdir("D:/My Drive/Wei/Patent")
ee_matcher=pd.read_csv("ee_list_other.csv")
stock_raw=pd.read_csv("PERMNO_dt.csv")

#Save ee_matcher,cleaner version of ee_list_other (1.ticker -> str data 2.Clean the ticker name that just generated)
#unify raw and matcher data
ee_matcher['ticker']=ee_matcher['ticker'].astype(str) #unify data type with stock_raw and patent_raw

tokens=pd.DataFrame(index=range(len(ee_matcher)),columns=range(1))

#clean the ee_matcher ticker name (eg. XXXX.XX-> XXXX)
tokens_list=[r.split('.') for r in ee_matcher['ticker']]
for i in range(len(ee_matcher)):
    x=tokens_list[i]
    tokens.at[i,0]=x[0]

tokens_list=[r.split('-') for r in tokens[0]]
for i in range(len(ee_matcher)):
    x=tokens_list[i]
    tokens.at[i,0]=x[0]

ee_matcher['ticker']=tokens[0] #save only the part before first . or -
x=patent_raw['ee_name'].drop_duplicates() #no duplicate ee_name_unclean
x=x.reset_index() #re index
x=x[['ee_name']]

ee_matcher['ee_name_unclean']=x  #add column ee_name_unclean to 

ee_matcher.to_csv('ee_matcher.csv',index=False)

#generate ee ticker txt file, used for finding PERMNO with CRSP data 
np.savetxt('ee_ticlist.txt', ee_matcher['ticker'].values, fmt='%s')
    

In [8]:
##Clean patent_raw stock_raw data:prepare for later merge
#1.ensure the ticker data type is str 
#2.decapitalize column name 'Ticker'
#3.change ee_name to ee_name_unclean

import numpy as np
import pandas as pd
import time
import os 

patent_raw=pd.read_csv("patent_latest_owner_dt.csv")
stock_raw=pd.read_csv("PERMNO_dt.csv")
patent_raw=patent_raw.rename(columns={'ee_name':'ee_name_unclean'}) #change ee_name to ee_name_unclean for merge
patent_raw=patent_raw.rename(columns={'or_name':'or_name_unclean'}) #change or_name to or_name_unclean for merge
stock_raw = stock_raw.rename(columns={'Ticker':'ticker'}) #decapitalize column name 'Ticker
stock_raw['ticker']=stock_raw['ticker'].astype(str) #ensure the ticker data type is str


patent_raw.to_csv('patent_raw.csv',index=False)
stock_raw.to_csv('stock_raw.csv',index=False)

In [2]:
###Match based on or_name
##Generate or_list (a unique list ready for matching with ticker)
import numpy as np
import pandas as pd
import os 
from cleanco import basename
import time
import requests


os.chdir("D:/My Drive/Wei/Patent")
patent_raw=pd.read_csv("patent_latest_owner_dt.csv")
stock_raw=pd.read_csv("PERMNO_dt.csv")

#Generate unique EE,OR,SecurityNm value
#EE & OR
or_name=patent_raw[['or_name']].drop_duplicates()

or_name=or_name.reset_index()

or_name=or_name[['or_name']]


#Processing company name (drop after the first comma, incase to long company name)
tokens=pd.DataFrame(index=range(len(or_name)),columns=range(1))
tokens_list=[r.split(',') for r in or_name['or_name']]

for i in range(len(or_name)):
    x=tokens_list[i]
    tokens.at[i,0]=x[0]

for i in range (len(or_name)):
    or_name.at[i,'or_name']=basename(tokens.iloc[i,0])
    
or_name['ticker']=''

or_name.to_csv('or_list.csv',index=False)

import numpy as np
import pandas as pd
import os 
from cleanco import basename
import time
import requests
os.chdir("D:/My Drive/Wei/Patent")

##Scraping with API and generate or_list_other (the or_list includes corresponding ticker)

or_name=pd.read_csv("or_list.csv")
def getTicker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

start = time.time()
for i in range (0,len(or_name)):
    try:
        company_name=or_name.iloc[i,0]
        or_name.at[i,'ticker']=getTicker(company_name)
    except:IndexError
    pass
    
    a=or_name.iloc[i,1]
    print(a)
    print(i)
    time.sleep(0.5) #Avoid reaching scraping times limit per minutes
    
end = time.time()
print(end - start)
or_name.to_csv('or_list_other.csv',index=False)

nan
0
nan
1
nan
2
nan
3
nan
4
nan
5
nan
6
nan
7
nan
8
nan
9
nan
10
COENF
11
HIO.AX
12
INCY
13
EMKR
14
nan
15
nan
16
COENF
17
QIU23.CMX
18
nan
19
nan
20
HILS
21
nan
22
nan
23
nan
24
nan
25
nan
26
VARIANTS-USD
27
nan
28
nan
29
nan
30
nan
31
nan
32
nan
33
nan
34
nan
35
nan
36
nan
37
KYN
38
nan
39
nan
40
nan
41
nan
42
nan
43
nan
44
HWKN
45
nan
46
nan
47
nan
48
nan
49
nan
50
nan
51
nan
52
nan
53
nan
54
nan
55
nan
56
nan
57
nan
58
nan
59
nan
60
46L.F
61
nan
62
nan
63
nan
64
DIS.MI
65
nan
66
nan
67
nan
68
nan
69
nan
70
nan
71
nan
72
nan
73
nan
74
nan
75
nan
76
nan
77
nan
78
nan
79
nan
80
nan
81
nan
82
nan
83
nan
84
nan
85
nan
86
nan
87
nan
88
nan
89
nan
90
nan
91
nan
92
nan
93
nan
94
MSFT
95
1222.HK
96
nan
97
VBDESAI.BO
98
nan
99
nan
100
COOK
101
nan
102
nan
103
AGARIND.NS
104
SWBI
105
000651.SZ
106
nan
107
ASCGX
108
PHH2.F
109
CARR
110
SU.PA
111
nan
112
nan
113
nan
114
nan
115
nan
116
nan
117
nan
118
nan
119
nan
120
PHH2.F
121
nan
122
nan
123
YUM
124
nan
125
nan
126
nan
127
nan
128
nan
129
n

In [6]:
##Further cleanning on the data:1.Generate or_matcher 2.generate or_ticlist.txt
import numpy as np
import pandas as pd
import time
import os 
os.chdir("D:/My Drive/Wei/Patent")
or_matcher=pd.read_csv("or_list_other.csv")
patent_raw=pd.read_csv("patent_latest_owner_dt.csv")

#Save or_matcher,cleaner version of or_list_other (1.ticker -> str data 2.Clean the ticker name that just generated)
#unify raw and matcher data
or_matcher['ticker']=or_matcher['ticker'].astype(str) #unify data type with stock_raw and patent_raw

tokens=pd.DataFrame(index=range(len(or_matcher)),columns=range(1))

#generate or_matcher
#clean the or_matcher ticker name (eg. XXXX.XX-> XXXX)
tokens_list=[r.split('.') for r in or_matcher['ticker']]
for i in range(len(or_matcher)):
    x=tokens_list[i]
    tokens.at[i,0]=x[0]

tokens_list=[r.split('-') for r in tokens[0]]
for i in range(len(or_matcher)):
    x=tokens_list[i]
    tokens.at[i,0]=x[0]

#include or_name_unclean
or_matcher['ticker']=tokens[0] #save only the part before first . or -
x=patent_raw['or_name'].drop_duplicates() #no duplicate ee_name_unclean
x=x.reset_index() #re index
x=x[['or_name']]

or_matcher['or_name_unclean']=x  #add column ee_name_unclean to 

or_matcher.to_csv('or_matcher.csv',index=False)

#generate ee ticker txt file, used for finding PERMNO with CRSP data 
np.savetxt('or_ticlist.txt', or_matcher['ticker'].values, fmt='%s')